<a href="https://colab.research.google.com/github/itimes-digital/mineracao-texto/blob/main/analise_de_texto_para_classificacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
import pandas as pd
import re

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
nltk.download('rslp')

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


True

In [4]:
base = pd.read_csv('sample_data/lista_classificada.csv', sep=';', engine='python')

In [5]:
base.head()

,Categoria,Comentario
0,"Cadastro, Login, Senha",Nao recebi email para validar o cadastro NÃO C...
1,"Cadastro, Login, Senha",Não consigo concluir o cadastro.
2,"Loja C&R, Estoque",Nunca realizei compras pelo app. Pois a forma ...
3,Entrega,"Muito demora pra entregar.Farmácia é urgência,..."
4,Entrega,"Péssimo no que se refere ao item ENTREGA, pois..."


In [6]:
base['Comentario'] = base['Comentario'].map(lambda x: re.sub('[,\.!?-]', '', x))
base['Comentario'] = base['Comentario'].map(lambda x: x.lower())

In [7]:
base.head()

,Categoria,Comentario
0,"Cadastro, Login, Senha",nao recebi email para validar o cadastro não c...
1,"Cadastro, Login, Senha",não consigo concluir o cadastro
2,"Loja C&R, Estoque",nunca realizei compras pelo app pois a forma d...
3,Entrega,muito demora pra entregarfarmácia é urgência q...
4,Entrega,péssimo no que se refere ao item entrega pois ...


In [8]:
stopwords = nltk.corpus.stopwords.words('portuguese')
#print(stopwords)

In [9]:
def removeStopWordsAplicandoStemmer(base):
    stemmer = nltk.stem.RSLPStemmer();
    frases = [];
    for i in range(len(base)):
        sem_stop = [str(stemmer.stem(p)) for p in base.loc[i,"Comentario"].split() if p not in stopwords]
        frases.append((sem_stop, base.loc[i,"Categoria"]))
    return frases;

In [10]:
frasesComStemming = removeStopWordsAplicandoStemmer(base);
#print(frasesComStemming)

In [11]:
def buscaPalavras(frases):
    todasPalavras = [];
    for (palavras, categoria) in frases:
        todasPalavras.extend(palavras);
    return todasPalavras;

In [12]:
palavras = buscaPalavras(frasesComStemming);
#print(palavras)

In [13]:
def buscaFrequencia(palavras):
    palavras = nltk.FreqDist(palavras);
    return palavras;

In [14]:
frequencia = buscaFrequencia(palavras);
#print(frequencia.most_common(50))

In [15]:
def buscaPalavrasUnicas(frequencia):
    freq = frequencia.keys();
    return freq;

In [16]:
palavrasUnicas = buscaPalavrasUnicas(frequencia);
#print(palavrasUnicas)

In [17]:
def extratorPalavras(documento):
    doc = set(documento);
    caracteristicas = {};
    for palavras in palavrasUnicas:
        caracteristicas['%s' % palavras] = (palavras in doc);
    return caracteristicas;

In [18]:
caracteristicasFrases = extratorPalavras({'am', 'nov', 'dia'});
#print(caracteristicasFrases)

In [36]:
baseCompleta = nltk.classify.apply_features(extratorPalavras, frasesComStemming);
#print(baseCompleta);

[({'nao': True, 'receb': True, 'email': True, 'valid': True, 'cadastr': True, 'cosig': True, 'acess': True, 'cont': True, 'consig': False, 'conclu': False, 'nunc': False, 'realiz': False, 'compr': False, 'app': False, 'poi': False, 'form': False, 'pag': False, 'limit': False, 'local': False, 'entreg': False, 'opç': False, 'produt': False, 'sit': False, 'atend': False, 'necess': False, 'dem': False, 'pra': False, 'entregarfarmác': False, 'urg': False, 'quer': False, 'facil': False, 'interf': False, 'saúd': False, 'péss': False, 'drogasil': False, 'ref': False, 'it': False, 'sempr': False, 'precis': False, 'medic': False, 'rotin': False, 'vai': False, 'man': False, 'escandal': False, 'doi': False, 'dia': False, 'uteisdesd': False, 'ontemqu': False, 'tent': False, 'cond': False, 'sair': False, 'cas': False, 'aparec': False, 'impossív': False, 'dev': False, 'ter': False, 'rápid': False, 'descont': False, 'vou': False, 'desinstal': False, 'us': False, 'dá': False, 'err': False, 'laboratóri'

In [20]:
classificador = nltk.NaiveBayesClassifier.train(baseCompleta);

In [21]:
print(classificador.show_most_informative_features(10))

Most Informative Features
                  cancel = True           Pedido : Lentid =     78.3 : 1.0
                    cart = True           Erro c : Entreg =     45.4 : 1.0
                     pix = True           Erro P : Lentid =     45.3 : 1.0
                   acess = True           Cadast : Lentid =     33.9 : 1.0
                     peg = True           Proble : Lentid =     30.5 : 1.0
                   retir = True           Proble : Lentid =     30.5 : 1.0
                  estoqu = True           Loja C : Lentid =     30.5 : 1.0
                  adiant = True           Proble : Lentid =     30.5 : 1.0
                  entreg = True           Entreg : Lentid =     30.2 : 1.0
                   códig = True           Erro P : Lentid =     29.0 : 1.0
None


### Montagem de uma frase para testar classificação.

In [22]:
testeFrase = 'Não aparece mais o Retirar na Loja!!! Desisti'

In [23]:
def aplicandoStemmerFrase(frase):
    stemmer = nltk.stem.RSLPStemmer();
    frases = [];
    for (palavras) in frase.split():
        com_stem_sem_stop = [p for p in palavras.split()];
        frases.append(str(stemmer.stem(com_stem_sem_stop[0])));
    return frases;

In [24]:
fraseComStemming = aplicandoStemmerFrase(testeFrase)
print(fraseComStemming)

['não', 'aparec', 'mais', 'o', 'retir', 'na', 'loja!!!', 'desist']


In [28]:
novaFrase = extratorPalavras(fraseComStemming);
#print(novaFrase)

{'nao': False, 'receb': False, 'email': False, 'valid': False, 'cadastr': False, 'cosig': False, 'acess': False, 'cont': False, 'consig': False, 'conclu': False, 'nunc': False, 'realiz': False, 'compr': False, 'app': False, 'poi': False, 'form': False, 'pag': False, 'limit': False, 'local': False, 'entreg': False, 'opç': False, 'produt': False, 'sit': False, 'atend': False, 'necess': False, 'dem': False, 'pra': False, 'entregarfarmác': False, 'urg': False, 'quer': False, 'facil': False, 'interf': False, 'saúd': False, 'péss': False, 'drogasil': False, 'ref': False, 'it': False, 'sempr': False, 'precis': False, 'medic': False, 'rotin': False, 'vai': False, 'man': False, 'escandal': False, 'doi': False, 'dia': False, 'uteisdesd': False, 'ontemqu': False, 'tent': False, 'cond': False, 'sair': False, 'cas': False, 'aparec': True, 'impossív': False, 'dev': False, 'ter': False, 'rápid': False, 'descont': False, 'vou': False, 'desinstal': False, 'us': False, 'dá': False, 'err': False, 'labora

In [26]:
print(classificador.classify(novaFrase))

Lentidão, Trava, Crash, Performance


In [27]:
distribuicao = classificador.prob_classify(novaFrase)
for classe in distribuicao.samples():
    print("%s: %f" % (classe, distribuicao.prob(classe)))

Cadastro, Login, Senha: 0.000000
Loja C&R, Estoque: 0.000000
Entrega: 0.004077
Preço, Prazo, Promoção, Stix: 0.000000
Erro PIX: 0.000002
Erro finalização: 0.000000
Cesta: 0.000000
Pedido cancelamento: 0.000000
Erro cartão de crédito: 0.000000
Lentidão, Trava, Crash, Performance: 0.995920
Busca: 0.000000
Problema não detalhado: 0.000000
Novas Funcionalidades: 0.000000


### Montagem dos dados novos para classificação

In [30]:
baseSemCategoria = pd.read_csv('sample_data/lista.csv', sep=';', engine='python')

In [33]:
baseSemCategoria['Comentario'] = baseSemCategoria['Comentario'].map(lambda x: re.sub('[,\.!?-]', '', x))
baseSemCategoria['Comentario'] = baseSemCategoria['Comentario'].map(lambda x: x.lower())

In [34]:
def removeStopWordsAplicandoStemmerSemCategoria(base):
    stemmer = nltk.stem.RSLPStemmer();
    frases = [];
    for i in range(len(base)):
        sem_stop = [str(stemmer.stem(p)) for p in base.loc[i,"Comentario"].split() if p not in stopwords];
        frases.append(sem_stop);
    return frases;

In [35]:
frasesComStemmingSemCategoria = removeStopWordsAplicandoStemmerSemCategoria(baseSemCategoria)
#print(frasesComStemmingSemCategoria)

[['nao', 'receb', 'email', 'valid', 'cadastr', 'cosig', 'acess', 'cont'], ['consig', 'conclu', 'cadastr'], ['nunc', 'realiz', 'compr', 'app', 'poi', 'form', 'pag', 'limit', 'local', 'entreg', 'opç', 'produt', 'app', 'sit', 'atend', 'necess'], ['dem', 'pra', 'entregarfarmác', 'urg', 'quer', 'facil', 'interf', 'saúd', 'necess', 'péss', 'drogasil', 'compr'], ['péss', 'ref', 'it', 'entreg', 'poi', 'sempr', 'precis', 'medic', 'rotin', 'opç', 'vai', 'man', 'escandal', 'pra', 'doi', 'dia', 'uteisdesd', 'ontemqu', 'tent', 'compr', 'poi', 'cond', 'sair', 'cas', 'aparec', 'entreg', 'impossív'], ['dev', 'ter', 'opç', 'entreg', 'rápid', 'ter', 'descont'], ['opç', 'entreg', 'dia'], ['vou', 'desinstal', 'app', 'us', 'sit', 'app', 'dá', 'descont', 'err', 'laboratóri', 'dá', 'opç', 'organiz', 'it', 'ord', 'preç', '(no', 'sit', 'opção)', 'viabiliz', 'compr', 'entreg', 'retir', 'loj', 'us', 's'], ['dem', 'demal', 'entreg'], ['problem', 'liber', 'ped', 'retir', 'loj', 'pag', 'via', 'pix', 'entreg', 'ped'

In [ ]:
baseCompletaSemClassificacao = nltk.classify.apply_features(extratorPalavras, frasesComStemmingSemCategoria);

In [ ]:
labels = classificador.classify_many(baseCompletaSemClassificacao)